## Contacts App
> Following [Hypermedia Systems book](https://hypermedia.systems)

In [1]:
#| default_exp contacts

In [2]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from monsterui.all import *
import json

In [3]:
#| export
app,rt,contacts,Contact = fast_app("data/contacts.db",hdrs=Theme.blue.headers(),
                                    id=int, first=str, last=str, phone=str, email=str, errors=str,
                                    pk="id", live=True)

In [4]:
#| export
if not contacts(): 
    with open("contacts.json","r") as f: data = json.load(f)
    contacts.insert_all(data)

### Table view

In [5]:
#| export
def header_render(col):
    """Render header for a table"""
    return Th(col.title() if col != "actions" else "", expand=True)

def action_buttons(contact):
    """Add buttons to view and edit contact"""
    return DivHStacked(
        Button(UkIcon("eye", cls="mr-2"), "View", cls=ButtonT.ghost, hx_get=f'/contacts/{contact.id}', hx_target='#modal-container', hx_swap="innerHTML"),
        Button(UkIcon("pencil", cls="mr-2"),"Edit", cls=ButtonT.ghost, hx_get=f'/contacts/{contact.id}/edit')
    )

def contacts_table(q=None):
    """Create a table with contacts applying a filter is query was provided"""
    rows = filter_contacts(q)
    return Table(
        Thead(
            Tr(*[Th(col) for col in ["First", "Last", "Phone", "Email", "Actions"]])
        ),
        Tbody(
            *[Tr(
                Td(c.first or ""),
                Td(c.last or ""),
                Td(c.phone or ""),
                Td(c.email or ""),
                Td(action_buttons(c))
            ) for c in rows]
        ),
        cls=(TableT.hover, TableT.divider, TableT.responsive),
        id="contacts-table"
    )

def filter_contacts(q=None):
    """Filter contacts that have `q` in first name, last name or email"""
    if not q: return contacts(order_by="id")
    q = q.lower()
    return [o for o in contacts(order_by="id")
            if q in (o.first or "").lower()
            or q in (o.last or "").lower()
            or q in (o.email or "").lower()]

### Detail view

In [6]:
#| export
def view_contact_modal(contact):
    """Detail view for a contact"""
    return Modal(
        ModalBody(
            DivVStacked(
                DivHStacked(Strong("First Name:"), P(contact.first or "-")),
                DivHStacked(Strong("Last Name:"), P(contact.last or "-")),
                DivHStacked(Strong("Phone:"), P(contact.phone or "-")),
                DivHStacked(Strong("Email:"), P(contact.email or "-")),
                cls="space-y-4"
            )
        ),
        header = ModalHeader(H3("Contact Details")),
        footer = ModalCloseButton("Close", cls=ButtonT.primary),
        id = "contact-modal",
        open=True  # This ensures the modal opens automatically
    )

### New Contacts

In [7]:
#| export
def contact_form(contact=None, action="/contacts/create"):
    """Reusable form that works both for new and edit"""
    return Form(
        Grid(
            LabelInput("First Name", id="first", value=contact.first if contact else ""),
            LabelInput("Last Name", id="last", value=contact.last if contact else ""),
            cols=2
        ),
        Grid(
            LabelInput("Phone", id="phone", value=contact.phone if contact else ""),
            LabelInput("Email", id="email", value=contact.email if contact else ""),
            cols=2
        ),
        DivRAligned(
            ModalCloseButton("Cancel", cls=ButtonT.ghost),
            Button("Save", cls=ButtonT.primary, type="submit")
        ),
        hx_post=action,
        hx_target="#modal-container"
    )

@rt("/contacts/new")
def get():
    """Create a modal for adding a contact"""
    return Modal(
        contact_form(),
        header=ModalHeader(H3("Add Contact")),
        id="contact-modal",
        open=True
    )

@rt("/dismiss-toast")
def get(): return Div(id="success-toast")  

@rt("/contacts/create")
def post(first:str, last:str, phone:str, email:str):
    """Create a new contact, show a success toast and update table with contacts"""
    new_contact = Contact(first=first, last=last, phone=phone, email=email)
    contacts.insert(new_contact)

    # create an out-of-band replacement for the contacts table
    updated_table = Div(
        contacts_table(),  # Get the refreshed table
        id="contacts-table",
        hx_swap_oob="true"  # Mark this for out-of-band swap
    )

    # Create a toast notification that appears in the center of the screen
    success_toast = Toast(
        DivLAligned(
            UkIcon("check-circle", cls='ml-2 text-success'),
            Span("Contact added successfully!")
        ),
        id="success-toast",
        alert_cls=AlertT.success,
        cls=(ToastHT.center, ToastVT.middle),
        hx_swap_oob="true",
        hx_get="/dismiss-toast",
        hx_trigger="load delay:3s",
        hx_target="#success-toast"
    )
    
    # Return both components
    return Div(
        updated_table,
        success_toast
    )

In [8]:
#| export
@rt("/")
def get(): return Redirect("/contacts")

@rt("/contacts/{id:int}")
def get(id:int): return view_contact_modal(contacts[id])


@rt("/contacts")
def get(q:str=None):
    search = Form(
        DivHStacked(
            Input(name="q", value=q, placeholder="Search contacts...",
                cls="w-full md:w-2/3 lg:w-1/2",
                hx_get="/contacts/search",
                hx_trigger="keyup changed delay:500ms",
                hx_target="#contacts-table",
                hx_include='[name="q"]'),
            Button("Search", type="submit")
       ),
       cls="mt-8"
    )
    return Container(page_heading, search, contacts_table(q),
                     add_button, Div(id="modal-container"), Div(id="success-toast"))

@rt("/contacts/search")
def get(q: str = ''): return contacts_table(q)

### Components

In [9]:
#| export
page_heading = Div(cls="space-y-2")(H1("Contacts"), P("Manage your contacts!", cls=TextPresets.muted_sm))

add_button = DivLAligned(
    Button(
        UkIcon("plus-circle", cls="mr-2"),
        "Add Contact",
        cls=ButtonT.primary,
        hx_get="/contacts/new",
        hx_target="#modal-container"
    ),
    cls="mb-4 mt-4"
)


In [10]:
# contacts.delete_where("id>=20")
# contacts()

In [11]:
server.stop()

In [12]:
server = JupyUvi(app)

In [13]:
HTMX()

In [14]:
# running at http://localhost:8000/

In [17]:
# from nbdev.export import nb_export
# nb_export('Contacts_v2.ipynb')